# Few shot learning

In [ ]:
from huggingface_hub import login  
login()

In [1]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, StoppingCriteriaList, StoppingCriteria

# Initialize Variables
model_name = "AI-Sweden-Models/gpt-sw3-1.3b"
device = "cuda:0" if torch.cuda.is_available() else "cpu" #in case i run this on my laptop or pc

In [ ]:

# Initialize Tokenizer & Model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()
model.to(device)

In [4]:
class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

In [6]:
def generate_text(prompt):
    stop_words = ["Besökare:", "Guide:"]
    stop_ids = [tokenizer.encode(stop_word, return_tensors="pt") for stop_word in stop_words]
    stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops = stop_ids)])
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    output = model.generate(
        inputs = input_ids,
        max_new_tokens = 50,
        do_sample=True,
        top_p=1, temperature=0.5,
        stopping_criteria=stopping_criteria
        )
    return tokenizer.decode(output[0] , skip_special_tokens=True)

prompt = "Följande är en konversation mellan en besökare på museet och en guide. Guiden arbetar på museet. Guiden är hjälpsam, informativ och mycket vänlig." \
            "Museet innehåller tre utställningar, den första utställningen heter 'Hitta nemo igen' av Hermann Gustafsson. Guiden ska hjälpa besökaren med frågor.\n\n"\
            "Besökare: Hej, jag är här för att se utställningen 'Hitta nemo igen', och jag har en fråga!\n" \
            "Guide: Hej, vad kul! Vad vill du veta om utställningen?\n" \
            "Besökare: Vet du vem som skapade utställningen?\n" \
            "Guide"
print(generate_text(prompt))         

Följande är en konversation mellan en besökare på museet och en guide. Guiden arbetar på museet. Guiden är hjälpsam, informativ och mycket vänlig.Museet innehåller tre utställningar, den första utställningen heter 'Hitta nemo igen' av Hermann Gustafsson. Guiden ska hjälpa besökaren med frågor.

Besökare: Hej, jag är här för att se utställningen 'Hitta nemo igen', och jag har en fråga!
Guide: Hej, vad kul! Vad vill du veta om utställningen?
Besökare: Vet du vem som skapade utställningen?
Guide: Ja, det är Hermann Gustafsson!
Besökare: Tack så mycket, vad bra!
Guide: Vad bra, då kan du gå och titta på utställningen!
Besökare: Tack så mycket!







